In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT 'CRIM' as column_name, COUNT() AS empty_count FROM boston WHERE (CRIM IS NULL OR CRIM = '')
    UNION
    SELECT 'ZN', COUNT() FROM boston WHERE (ZN IS NULL OR ZN = '')
    UNION
    SELECT 'INDUS', COUNT() FROM boston WHERE (INDUS IS NULL OR INDUS = '')
    UNION
    SELECT 'CHAS', COUNT() FROM boston WHERE (CHAS IS NULL OR CHAS = '')
    UNION
    SELECT 'NOX', COUNT() FROM boston WHERE (NOX IS NULL OR NOX = '')
    """,
    con,
)

,column_name,empty_count
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT 'CRIM' as column_name, COUNT(DISTINCT CRIM) AS unique_count FROM boston
    UNION
    SELECT 'ZN', COUNT(DISTINCT ZN) FROM boston
    UNION
    SELECT 'INDUS', COUNT(DISTINCT INDUS) FROM boston
    UNION
    SELECT 'CHAS', COUNT(DISTINCT CHAS) FROM boston
    UNION
    SELECT 'NOX', COUNT(DISTINCT NOX) FROM boston
    """,
    con,
)

,column_name,unique_count
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
pd.read_sql(
    """
SELECT
    column_name
FROM 
    (SELECT 
        'CRIM' AS column_name, 
        (SELECT
            AVG(CRIM)
         FROM
            (SELECT
                 CRIM
             FROM
                 boston
             ORDER BY
                 CRIM
             LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
             OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)
             )
          ) = MIN(CRIM) AS flag
    FROM
        boston
    UNION
        SELECT 
            'ZN' AS column_name, 
        (SELECT
            AVG(ZN)
         FROM
             (SELECT
                 ZN
              FROM
                  boston
              ORDER BY
                  ZN
              LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
              OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)
              )
          ) = MIN(ZN) AS flag
    FROM 
        boston
    UNION
        SELECT 
            'INDUS' AS column_name, 
            (SELECT
                AVG(INDUS)
             FROM 
                 (SELECT
                      INDUS
                  FROM
                      boston
                  ORDER BY
                      INDUS
                  LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                  OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)
                 )
              ) = MIN(INDUS) AS flag
        FROM 
            boston
    UNION
        SELECT 
        'CHAS' AS column_name, 
        (SELECT 
            AVG(CHAS)
         FROM 
             (SELECT
                 CHAS
               FROM
                   boston
               ORDER BY
                   CHAS
               LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
               OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)
              )
          ) = MIN(CHAS) AS flag
        FROM 
            boston
    UNION
        SELECT 
            'NOX' AS column_name, 
            (SELECT
                AVG(NOX)
             FROM 
                 (SELECT
                     NOX
                  FROM
                      boston
                  ORDER BY
                      NOX
                  LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                  OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)
                  )
             ) = MIN(NOX) AS flag
        FROM
            boston
    )
     WHERE
         flag = 1
    """,
    con,
)

,column_name
0,CHAS
1,ZN


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
pd.read_sql(
    """
    WITH 
        highest_price AS 
            (SELECT 
                AVG(RM) AS avg_high
             FROM 
                 boston
             GROUP BY
                 MEDV
             ORDER BY
                 MEDV DESC
             LIMIT
                 1
            ),
        min_25 AS 
            (SELECT 
                25 AS num_houses,
                AVG(RM) AS rm_avg,
                avg_high AS rm_avg_h,
                avg_high - AVG(RM) AS diff
             FROM
                (SELECT
                    RM,
                    MEDV
                 FROM
                     boston
                 ORDER BY
                     MEDV ASC
                 LIMIT
                     25
                 ),
                 highest_price
              ),
        min_50 AS 
            (SELECT 
                50 AS num_houses,
                AVG(RM) AS rm_avg,
                avg_high AS rm_avg_h,
                avg_high - AVG(RM) AS diff
             FROM
                (SELECT
                    RM,
                    MEDV
                 FROM
                     boston
                 ORDER BY
                     MEDV ASC
                 LIMIT
                     50
                 ),
                 highest_price
              ),
        min_100 AS 
            (SELECT 
                100 AS num_houses,
                AVG(RM) AS rm_avg,
                avg_high AS rm_avg_h,
                avg_high - AVG(RM) AS diff
             FROM
                (SELECT
                    RM,
                    MEDV
                 FROM
                     boston
                 ORDER BY
                     MEDV ASC
                 LIMIT
                     100
                 ),
                 highest_price
              ),
       min_200 AS 
            (SELECT 
                200 AS num_houses,
                AVG(RM) AS rm_avg,
                avg_high AS rm_avg_h,
                avg_high - AVG(RM) AS diff
             FROM
                (SELECT
                    RM,
                    MEDV
                 FROM
                     boston
                 ORDER BY
                     MEDV ASC
                 LIMIT
                     200
                 ),
                 highest_price
              ),
        min_300 AS 
            (SELECT 
                300 AS num_houses,
                AVG(RM) AS rm_avg,
                avg_high AS rm_avg_h,
                avg_high - AVG(RM) AS diff
             FROM
                (SELECT
                    RM,
                    MEDV
                 FROM
                     boston
                 ORDER BY
                     MEDV ASC
                 LIMIT
                     300
                 ),
                 highest_price
              )
    
    SELECT * FROM min_25
    UNION ALL
    SELECT * FROM min_50
    UNION ALL
    SELECT * FROM min_100
    UNION ALL
    SELECT * FROM min_200
    UNION ALL
    SELECT * FROM min_300
    """,
    con,
)

,num_houses,rm_avg,rm_avg_h,diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


#### Вывод к 4: количество комнат влияет на стоимость недвижимости, но не является основным фактором.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [11]:
pd.read_sql(
    """
    SELECT
        q.*
    FROM 
        (SELECT
            LSTAT,
            MEDV,
            RANK() OVER(ORDER BY LSTAT ASC) AS LSTAT_RANK
         FROM
            boston     
    ) AS q 
    WHERE
        q.MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)

,LSTAT,MEDV,LSTAT_RANK
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


#### Вывод по 5: налог на недвижимость в США зависит от её рыночной стоимости, и как только район становится популярным и стоимость недвижимости начинает расти, люди с малым доходом уже не могут платить увеличившийся налог и им приходится продавать свою недвижимость и переезжать в районы с меньшей стоимостью недвижимости. Получается своего рода сегрегация.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
    SELECT
        AVG(MEDV) as AVG_PRICE,
        (CASE 
            WHEN
                CHAS == 0
            THEN 
                'нет' 
             ELSE 
                'да' 
         END) AS ГРАНИЧИТ_С_РЕКОЙ
    FROM
        boston
    GROUP BY
        CHAS
    """,
    con,
)

,AVG_PRICE,ГРАНИЧИТ_С_РЕКОЙ
0,22.093843,нет
1,28.440000,да


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
pd.read_sql(
    """
    SELECT
        q.column_name
    FROM 
        (
        SELECT 
            'CRIM' AS column_name, 
            (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) > AVG(CRIM) AS flag
        FROM
            boston
        UNION
        SELECT 
            'ZN' AS column_name, 
            (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) > AVG(ZN) AS flag
        FROM
            boston
        UNION
        SELECT 
            'INDUS' AS column_name, 
            (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) > AVG(INDUS) AS flag
        FROM
            boston
        UNION
        SELECT 
            'NOX' AS column_name, 
            (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) > AVG(NOX) AS flag
        FROM
            boston
        ) AS q
    WHERE
        flag = 1
    """,
    con,
)

,column_name
0,INDUS
1,NOX


##### Вывод по 7: доля акров неторгового бизнеса на город (INDUS) выше среднего - промышленные предприятия расположены по берегам рек (логично, т.к. для производств требуется большое количество воды), концентрация оксидов азота (частей на 10 миллионов) (NOX) выше среднего - эти предприятия не простаивают 

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
pd.read_sql(
    """
    WITH
        q as (SELECT INDUS, NTILE(10) OVER (ORDER BY INDUS) AS percentile FROM boston),
        s as (SELECT NOX, NTILE(10) OVER (ORDER BY NOX) AS percentile FROM boston)
    SELECT
        INDUS,
        NOX,
        q.percentile
    FROM
        q, s
    WHERE
        q.percentile = s.percentile
    GROUP BY
        q.percentile
    """,
    con,
)

,INDUS,NOX,percentile
0,0.46,0.3850,1
1,2.93,0.4280,2
2,4.39,0.4429,3
3,5.96,0.4720,4
4,7.38,0.5070,5
5,9.69,0.5380,6
6,13.89,0.5800,7
7,18.10,0.6090,8
8,18.10,0.6710,9
9,19.58,0.7130,10


##### Вывод по 8: прослеживается прямая зависимость концентрации оксидов азота (NOX) и долей акров неторгового бизнеса на город (промышленностью).